# Parameter Scan

## Imports and Get Example model
Here we use the Zi 2012 model of TGF$\beta$ dynamics

In [10]:
import site
site.addsitedir('//home/b3053674/Documents/pycotools')
from pycotools import model, tasks
from pycotools.Tests import test_models
import os


## get string model from test_models
zi_model_string = test_models.TestModels().zi_model()

## get a working directory. Change this to change this to wherever you like
directory = r'/home/b3053674/Documents/pycotools/ZiModel'

## choose path to zi model
zi_path = os.path.join(directory, 'zi2012.cps')

##write model to file
with open(zi_path, 'w') as f:
    f.write(zi_model_string)
    
## check file exists
if not os.path.isfile(zi_path):
    raise Exception
    
zi = model.Model(zi_path)

Copasi supports three types of scan: 

## Parameter Scan
Set up a scan subtask with a single parameter being systematically varied. Run the scan via CopasiSE and have the output sqved to file.

In [7]:
zi = tasks.Scan(
    zi,scan_type = 'scan',
    subtask = 'time_course',
    report_type = 'time_course',
    report_name = 'ParameterScanOfTimeCourse.txt',
    variable = 'Smad3c',
    minimum = 1,
    maximum = 20,
    number_of_steps = 8,
    run=True
).model

pycotools.tasks:WARNING:1775:    output_in_subtask is False. Setting to True in order to have data from all scans output to file


Note that the zi model is reassigned to the version of the model within the Scan instance. We can then `save`, `open` or carry the configured model forward to more analyses.

## 2 Way Parameter Scan
By default, scan tasks are removed before setting up a new scan. To set up multiple scans, set  `clear_scans` to `False` in all but the first scan. 

In [11]:
## setup first parameter scan
zi = tasks.Scan(
    zi, scan_type='scan', subtask='time_course',
    report_type='time_course', report_name='ParameterScanOfTimeCourse.txt',
    variable='Smad3c', minimum=1, maximum=10, number_of_steps=10
).model

## setup second parameter scan. Note clear_scans = False. 
tasks.Scan(
    zi, scan_type='scan', variable='Smad3n', minimum=1,
    maximum=100, number_of_steps=10, clear_scans=False, run=True, save=True
).model
zi.open()

pycotools.tasks:WARNING:1775:    output_in_subtask is False. Setting to True in order to have data from all scans output to file
pycotools.tasks:WARNING:1775:    output_in_subtask is False. Setting to True in order to have data from all scans output to file


An arbitrary number of scans can be setup this way. Note that this time we used the `save=True` argument to save the model inplace. 

## Repeat Scan Items
Repeat scans are useful for running multiple parameter estimations and for running stochastic time courses. 

### Setup repeat parameter estimation task

In [12]:
## Assume Parameter Estimation task already configured
tasks.Scan(
    zi, scan_type='repeat', subtask='parameter_estimation',
    report_type='parameter_estimation', report_name='estimations.txt',
    number_of_steps = 10, run=False, save=True 
)

pycotools.tasks:DEBUG:1734:    {'1': 'scan', '0': 'repeat', '2': 'random_sampling'}


Scan(scan_type='repeat', subtask='parameter_estimation', variable='Smad3n', report_type='parameter_estimation', report_name='estimations.txt', number_of_steps='10', minimum=0.01, maximum=100)

Notes:
-  Subtask is configured to parameter_estimation. The parameter estimation must already be appropriately configured
-  Here we setup 10 estimations and save the outut to a text file in the current directory called estimations.txt
-  run is set to False. Set to True to run the scan task
-  Zi model gets replaced with the version from within Scan

### Setup repeat stochastic time courses

In [ ]:
## Assume Time course task already configured to run a with stochastic solver
tasks.Scan(
    zi, scan_type='repeat', subtask='time_course',
    report_type='time_course', report_name='stochastic_timecourse.txt',
    number_of_steps = 100, run=False, save=True
)

Notes
-  A stochastic time course must already be configured for the multiple scans to run. 